In [1]:
import input_data
import numpy
import tensorflow as tf

In [2]:
mnist = input_data.read_data_sets("./tf_mnist", one_hot=True)

Extracting ./tf_mnist\train-images-idx3-ubyte.gz
Extracting ./tf_mnist\train-labels-idx1-ubyte.gz
Extracting ./tf_mnist\t10k-images-idx3-ubyte.gz
Extracting ./tf_mnist\t10k-labels-idx1-ubyte.gz


In [3]:
x_train = tf.placeholder(tf.float32, [None, 784])
y_train = tf.placeholder(tf.float32, [None, 10])

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [5]:
def bias_variable(shape):
#    initial = tf.constant(0.1, shape=shape)
    initial = tf.zeros(*shape)
    return tf.Variable(initial)

In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [7]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
        strides=[1, 2, 2, 1], padding='SAME')

In [8]:
def deepnn(x):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    w1 = weight_variable([5, 5, 1, 32])
    b1 = bias_variable([32])
    h1_conv = tf.nn.relu( conv2d(x_image, w1) + b1)
    h1 = max_pool_2x2(h1_conv)
    
    w2 = weight_variable([5, 5, 32, 64])
    b2 = bias_variable([64])
    h2_conv = tf.nn.relu( conv2d(h1, w2) + b2)
    h2 = max_pool_2x2(h2_conv)
    
    w_fc1 = weight_variable([7*7*64, 1024])
    b_fc1 = bias_variable([1024])
    h2_flat = tf.reshape(h2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu( tf.matmul(h2_flat, w_fc1) + b_fc1 )
    
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    w_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    
    y_conv = tf.matmul(h_fc1_drop, w_fc2) + b_fc2
    return y_conv, keep_prob

In [9]:
# Build the graph for the deep net
y_conv, keep_prob = deepnn(x_train)

In [10]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_train, 1))
accuracy = tf.reduce_mean( tf.cast(correct_prediction, tf.float32 ) )

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval( feed_dict = {
                x_train: batch[0], y_train: batch[1], keep_prob: 1.0 })
            print ('step %d, train accuracy %g' % (i, train_accuracy))
        train_step.run( feed_dict = {
            x_train: batch[0], y_train: batch[1], keep_prob: 0.5 })
    print ('test accuracy %g' % accuracy.eval(feed_dict = {
        x_train: mnist.test.images, y_train: mnist.test.labels, keep_prob: 1.0}))

step 0, train accuracy 0.02
step 100, train accuracy 0.8
step 200, train accuracy 0.94
step 300, train accuracy 0.88
step 400, train accuracy 1
step 500, train accuracy 0.94
step 600, train accuracy 0.96
step 700, train accuracy 0.94
step 800, train accuracy 0.94
step 900, train accuracy 1
step 1000, train accuracy 0.94
step 1100, train accuracy 0.96
step 1200, train accuracy 0.94
step 1300, train accuracy 1
step 1400, train accuracy 1
step 1500, train accuracy 0.96
step 1600, train accuracy 1
step 1700, train accuracy 1
step 1800, train accuracy 0.98
step 1900, train accuracy 1
step 2000, train accuracy 0.98
step 2100, train accuracy 0.98
step 2200, train accuracy 1
step 2300, train accuracy 1
step 2400, train accuracy 1
step 2500, train accuracy 0.98
step 2600, train accuracy 0.98
step 2700, train accuracy 1
step 2800, train accuracy 1
step 2900, train accuracy 1
step 3000, train accuracy 1
step 3100, train accuracy 0.96
step 3200, train accuracy 1
step 3300, train accuracy 1
step 34